In [1]:
!pip install torch torchvision
!pip install -U finance-datareader


In [2]:
import pandas as pd
import pandas_datareader as pdr
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
import time
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
x_frames = 7
'''
start = datetime.datetime.now() - datetime.timedelta(days=(x_frames*3))
end = datetime.date.today()
df_du = pdr.DataReader('POILDUBUSDM', 'fred', start=datetime.datetime.now() - datetime.timedelta(days=(100)), end=end)#석유
df_gold = pdr.DataReader('GOLDAMGBD228NLBM', 'fred', start=start, end=end)#금값
df_kospi = fdr.DataReader('KS11', start, end)#코스피
df_dollar = fdr.DataReader('USD/KRW', start, end)#원달러환율

df_kospi['gold'] = df_gold
df_kospi['dollar'] = df_dollar['Close']
#oil 채우기
du = [df_du.values[-1] for i in range(x_frames)]

#gold 채우기
df_kospi = df_kospi.fillna(value=-1)
for i,val in enumerate(df_kospi['gold']):
    if val == -1:
        df_kospi['gold'][i] = df_kospi['gold'][i-1]
df_kospi = df_kospi.tail(x_frames)
df_kospi['oil'] = pd.DataFrame(data=du, index=df_kospi.index, columns=['oil'])
'''

"\nstart = datetime.datetime.now() - datetime.timedelta(days=(x_frames*3))\nend = datetime.date.today()\ndf_du = pdr.DataReader('POILDUBUSDM', 'fred', start=datetime.datetime.now() - datetime.timedelta(days=(100)), end=end)#석유\ndf_gold = pdr.DataReader('GOLDAMGBD228NLBM', 'fred', start=start, end=end)#금값\ndf_kospi = fdr.DataReader('KS11', start, end)#코스피\ndf_dollar = fdr.DataReader('USD/KRW', start, end)#원달러환율\n\ndf_kospi['gold'] = df_gold\ndf_kospi['dollar'] = df_dollar['Close']\n#oil 채우기\ndu = [df_du.values[-1] for i in range(x_frames)]\n\n#gold 채우기\ndf_kospi = df_kospi.fillna(value=-1)\nfor i,val in enumerate(df_kospi['gold']):\n    if val == -1:\n        df_kospi['gold'][i] = df_kospi['gold'][i-1]\ndf_kospi = df_kospi.tail(x_frames)\ndf_kospi['oil'] = pd.DataFrame(data=du, index=df_kospi.index, columns=['oil'])\n"

In [13]:
class predict_Dataset(Dataset):
    
    def __init__(self, symbol, x_frames):
        
        self.symbol = symbol
        self.x_frames = x_frames
        
        self.start = datetime.datetime.now() - datetime.timedelta(days=(x_frames*3))
        self.end = datetime.date.today()

        self.scaler = MinMaxScaler()
        self.data = fdr.DataReader(self.symbol, self.start, self.end)#주식 6가지 특징, 고,시,저,종,거,변
        
        if len(self.data.index) == 0:
            print(self.symbol,' is zero')
            return None
        self.y_max = max(self.data['Close'])
        self.y_min = min(self.data['Close'])
        self.data = self.data.tail(x_frames)
        self.data = pd.DataFrame(self.scaler.fit_transform(self.data))
        
        
    def __len__(self):
        return len(self.data) - x_frames + 1
    
    def __getitem__(self, idx):
        data = self.data[[0,1,2,3,4]]
        X = data.values
        #X = data[:]
        
        return X, self.y_max, self.y_min

In [6]:
class GRU(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(GRU, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        self.gru = nn.GRU(self.input_dim, self.hidden_dim*2, self.num_layers)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers)
        self.regressor = self.make_regressor()

    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim//2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim//2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        y_pred = self.regressor(x[-1].view(self.batch_size, -1))
        return y_pred

In [7]:
#주식정보들
Symbols = ['005930','000660','035420','207940','051910','005935','005380','068270','035720','006400','051900','012330','000270','028260']
'''
with open('/content/gdrive/My Drive/Symbols.txt', 'r') as f:
    data = f.readline()
    Symbols = data.split(',')
print(len(Symbols))
'''
'''
005930   삼성전자
000660   SK하이닉스
035420   NAVER
207940   삼성바이오로직스
051910   LG화학
005935   삼성전자우
005380   현대차
068270   셀트리온
035720   카카오
006400   삼성SDI
051900   LG생활건강
012330   현대모비스
000270   기아차
028260   삼성물산
'''

'\n005930   삼성전자\n000660   SK하이닉스\n035420   NAVER\n207940   삼성바이오로직스\n051910   LG화학\n005935   삼성전자우\n005380   현대차\n068270   셀트리온\n035720   카카오\n006400   삼성SDI\n051900   LG생활건강\n012330   현대모비스\n000270   기아차\n028260   삼성물산\n'

In [8]:
batch_size = 1
input_dim = 5
hidden_dim = 64
num_layers = 2
input_frame = 7
output_frame = 7
dropout = 1.0
use_bn = True

model = GRU(input_dim,hidden_dim,output_frame,num_layers,batch_size,dropout,use_bn)
modelPath = '/content/gdrive/My Drive/GRUmodel/GRU_5_model.pt'
model.load_state_dict(torch.load(modelPath))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

GRU(
  (gru): GRU(5, 128, num_layers=2)
  (gru2): GRU(128, 64, num_layers=2)
  (regressor): Sequential(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=1.0, inplace=False)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=7, bias=True)
  )
)

In [14]:
predicts = {}
i = 0
for sb in Symbols:
    i += 1
    if i % 100 == 0:
        print(i)
    pset = predict_Dataset(sb,input_frame)
    model.eval()
    loader = DataLoader(pset,batch_size,shuffle=False)
    for X, y_max, y_min in loader:
        X = X.transpose(0, 1).float().to(device)
        y_pred = model(X)
        y_pred = y_pred.view(-1)
        predicts[sb] = int(y_pred[-1]*(y_max.to(device)-y_min.to(device))+y_min.to(device))


In [15]:
print(predicts)
'''
005930   삼성전자
000660   SK하이닉스
035420   NAVER
207940   삼성바이오로직스
051910   LG화학
005935   삼성전자우
005380   현대차
068270   셀트리온
035720   카카오
006400   삼성SDI
051900   LG생활건강
012330   현대모비스
000270   기아차
028260   삼성물산
'''


{'005930': 58639, '000660': 85342, '035420': 293876, '207940': 689667, '051910': 635953, '005935': 53002, '005380': 173989, '068270': 256892, '035720': 360477, '006400': 423212, '051900': 1561707, '012330': 227351, '000270': 46206, '028260': 102420}


'\n005930   삼성전자\n000660   SK하이닉스\n035420   NAVER\n207940   삼성바이오로직스\n051910   LG화학\n005935   삼성전자우\n005380   현대차\n068270   셀트리온\n035720   카카오\n006400   삼성SDI\n051900   LG생활건강\n012330   현대모비스\n000270   기아차\n028260   삼성물산\n'

In [ ]:
'''
!pip install pymysql
import pymysql
'''

In [ ]:
conn = pymysql.connect(host='jukerdb.cwhsnjoqybdo.ap-northeast-2.rds.amazonaws.com', user='admin', password='',
                       db='WEB', charset='utf8')
 
# Connection 으로부터 Cursor 생성
curs = conn.cursor()
 

for sb in result:
    #table : stockpreidct
    #symbol, value, type, day
    # SQL문 실행
    sql = """INSERT INTO stockpredict VALUES(%s,%s,%s,NOW())"""
    val = (sb,result[sb],'L')
    curs.execute(sql,val)
    conn.commit()

sql = "select * from stockpredict"
curs.execute(sql)
rows = curs.fetchall()
print(rows)     # 전체 rows

# Connection 닫기
conn.close()